# SealNet Detection Pipeline -- Training / Validation
---

This jupyter notebook will go through training CNN within the SealNet pipeline. Requires a training set generated on the [training_set_generation](https://github.com/iceberg-project/Seals/blob/paper/SealNet_code/training_set_generation.ipynb) notebook.




## Table of contents
---
* [Getting started](#intro)
    * [Setup](#setup)
    * [Visualize training set](#vis_imgs)
* [Pipeline 1 - Heatmap](#1)
    * [Training](#1T)
    * [Validation](#1V)
* [Pipeline 2.1 - Heatmap + count](#2.1)
    * [Training](#2.1T)
    * [Validation](#2.1V)
* [Pipeline 2.2 - Heatmap + occupancy](#2.2)
    * [Training](#2.2T)
    * [Validation](#2.2V)
* [Pipeline 3 - Heatmap + count + occupancy](#3)
    * [Training](#3T)
    * [Validation](#3V)

## Getting started<a name="intro"></a>
---

If you followed the *training_set_generation* jupyter notebook (also present in this repo), you should have training sets generated and hyperparameter sets to try out, and be ready to search for a best performing seal detection pipeline.  Output files in this repository are organized as follows: *'./{dest_folder}/{pipeline}/{model_settings}/{model_settings}_{file}'*

### Setup environment<a name="setup"></a>

Before training and validating model/hyperparameter combinations inside the pipelines, we need to load the required python modules and a few global variables.

In [16]:
# import required packages
import os
import rasterio
import pandas as pd
import numpy as np
import operator
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from functools import reduce
from utils.model_library import * 

%matplotlib inline
mpl.rcParams['figure.dpi']= 400

# destination folder for saved models and model stats
dest_folder = 'saved_models_all_train'

### Visualizing training images (Optional)<a name="vis_imgs"></a>

To get a better sense for what the training set is like, the next cell will display a few random images from the training classes. Displayed images are extracted from a pool of ~75000 training images. 

In [ ]:
# save class names
class_names = sorted([subdir for subdir in os.listdir('./training_sets/training_set_vanilla/training')])
# store images
images = []

# loop over labels
for label in class_names:
    for path, _, files in os.walk('./training_sets/training_set_vanilla/training/{}'.format(label)):
        files = np.random.choice(files, 5)
        for filename in files:
            images.append(np.asarray(Image.open(os.path.join(path, filename))))

images = np.array(images)

# display images 
ncols=len(class_names)
nindex, height, width, intensity = images.shape
nrows = nindex // ncols
# check if rows and columns can fit the number of images
assert nindex == nrows * ncols
result = (images.reshape(nrows, ncols, height, width, intensity)
          .swapaxes(1,2)
          .reshape(height*nrows, width*ncols, intensity))

plt.imshow(result)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)
plt.show()
    


## Pipeline 1 - Heatmap models <a name="1"></a>
---

Heatmap models work by using semantic segmentation to generate a pixel-wise probability of a cell being a seal centroid. With a matrix of probabilities, we obtain a count by applying a sigmoid transform to it, thresholding and adding over all cells. Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image.


### Training<a name="1T"></a>

The following cell trains heatmap based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [20]:
# switch pipeline
pipeline = 'Heatmap'

# generate model combinations
combinations_1 = {'model_architecture': ['Unet'],
                  'training_dir': ['training_set_vanilla'],
                  'hyperparameter_set': ['A']}

# read as a DataFrame
combinations_1 = pd.DataFrame(combinations_1)
                    
# create folders for resulting files
for row in combinations_1.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]              
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 


We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of required arguments can be displayed by running the cell below.

In [ ]:
%run train_sealnet.py -h

In [21]:
# iterate over combinations
for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st \
                             --output_name=$out --models_folder=$dest_folder --all_train=1 
                              
      

Unet_ts-vanilla was already trained

training Unet_ts-vanilla

Epoch 1/150
----------

training 

heatmap loss: 0.40820748625963016
count loss: 682.252008949313
occupancy loss: 0.0

validation 

Epoch 2/150
----------

training 

heatmap loss: 0.2538380302605208
count loss: 24.752167872092652
occupancy loss: 0.0

validation 

Epoch 3/150
----------

training 

heatmap loss: 0.2506173912442491
count loss: 15.78364265266141
occupancy loss: 0.0

validation 

Epoch 4/150
----------

training 

heatmap loss: 0.24677915929186217
count loss: 12.490176485495558
occupancy loss: 0.0

validation 

Epoch 5/150
----------

training 

heatmap loss: 0.25076327017201583
count loss: 11.466143883899155
occupancy loss: 0.0

validation 

Epoch 6/150
----------

training 

heatmap loss: 0.24099451639609892
count loss: 9.879825623671346
occupancy loss: 0.0

validation 

Epoch 7/150
----------

training 

heatmap loss: 0.24858406070689051
count loss: 9.429147746938154
occupancy loss: 0.0

validation 

Epoch 

heatmap loss: 0.24131735509491498
count loss: 4.144859415092569
occupancy loss: 0.0

validation 

Epoch 63/150
----------

training 

heatmap loss: 0.248782427880832
count loss: 4.131279735541364
occupancy loss: 0.0

validation 

Epoch 64/150
----------

training 

heatmap loss: 0.2465212284278003
count loss: 4.182259484547913
occupancy loss: 0.0

validation 

Epoch 65/150
----------

training 

heatmap loss: 0.2418804252586656
count loss: 3.9074244522807975
occupancy loss: 0.0

validation 

Epoch 66/150
----------

training 

heatmap loss: 0.24580031807043123
count loss: 4.194481646587807
occupancy loss: 0.0

validation 

Epoch 67/150
----------

training 

heatmap loss: 0.25010762766014877
count loss: 3.436069905476672
occupancy loss: 0.0

validation 

Epoch 68/150
----------

training 

heatmap loss: 0.24833407546275316
count loss: 3.8873940581297806
occupancy loss: 0.0

validation 

Epoch 69/150
----------

training 

heatmap loss: 0.2405783355262645
count loss: 3.7818416987330985


heatmap loss: 0.24344194237195543
count loss: 1.6848267712307656
occupancy loss: 0.0

validation 

Epoch 124/150
----------

training 

heatmap loss: 0.23805378102411107
count loss: 1.4458632722051703
occupancy loss: 0.0

validation 

Epoch 125/150
----------

training 

heatmap loss: 0.24491393160756889
count loss: 1.6549200243398015
occupancy loss: 0.0

validation 

Epoch 126/150
----------

training 

heatmap loss: 0.24680131997327157
count loss: 1.3937851772276146
occupancy loss: 0.0

validation 

Epoch 127/150
----------

training 

heatmap loss: 0.24284640952353104
count loss: 1.4704871015606495
occupancy loss: 0.0

validation 

Epoch 128/150
----------

training 

heatmap loss: 0.26024169858262197
count loss: 1.5397580786278073
occupancy loss: 0.0

validation 

Epoch 129/150
----------

training 

heatmap loss: 0.2395113318557732
count loss: 1.4007403062526762
occupancy loss: 0.0

validation 

Epoch 130/150
----------

training 

heatmap loss: 0.24403745763087714
count loss: 1.4

### Validation<a name="1V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [58]:
# iterate over combinations
for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run training
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder


validating Unet_ts-vanilla

heatmap loss: 0.2017389613420684
count loss: 0.2531767993075707
occupancy loss: 0.0
Validation complete in 0.0h 0m 59s


## Pipeline 2.1 - Heatmap + count<a name="1.1"></a>
---

This pipeline will also generate a seal intensity heatmap like the previous one, but it does not add over pixels to get a count. Instead, Heatmap + count models have a specialized branch to get a count by regression. Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image

### Training<a name="2.1T"></a>

The following cell trains heatmap + count based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [11]:
# switch pipeline
pipeline = 'Heatmap-Cnt'

# generate model combinations
combinations_21 = {'model_architecture': ['UnetCntWRN'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['F']}       

# read as a DataFrame
combinations_21 = pd.DataFrame(combinations_21)
                    

# create folders for resulting files
for row in combinations_21.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]               
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [49]:
# iterate over combinations
for row in combinations_21.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()

    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st \
                             --output_name=$out --models_folder=$dest_folder
                              
      


training UnetCntWRN_ts-vanilla

training_set_vanilla UnetCntWRN F UnetCntWRN_ts-vanilla saved_models
Epoch 1/75
----------

training 

heatmap loss: 1.0941149730974762
count loss: 0.5377885517448784
occupancy loss: 0.0

validation 

heatmap loss: 0.3136393334413029
count loss: 0.17241308105041162
occupancy loss: 0.0
training time: 0.0h 7m 26s

Epoch 2/75
----------

training 

heatmap loss: 0.6453341031173254
count loss: 0.3592416110453977
occupancy loss: 0.0

validation 

heatmap loss: 0.29030913505801537
count loss: 0.12916605050821137
occupancy loss: 0.0
training time: 0.0h 14m 57s

Epoch 3/75
----------

training 

heatmap loss: 0.6527025189674509
count loss: 0.2920381472443365
occupancy loss: 0.0

validation 

heatmap loss: 0.28657869291819915
count loss: 0.07131773855862936
occupancy loss: 0.0
training time: 0.0h 22m 28s

Epoch 4/75
----------

training 

heatmap loss: 0.6442108458475418
count loss: 0.23017428475634152
occupancy loss: 0.0

validation 

heatmap loss: 0.2841309070

heatmap loss: 0.6404674243033522
count loss: 0.07977807474294567
occupancy loss: 0.0

validation 

heatmap loss: 0.2803368893325233
count loss: 0.033641716743393404
occupancy loss: 0.0
training time: 4.0h 14m 2s

Epoch 35/75
----------

training 

heatmap loss: 0.6336624196455151
count loss: 0.07324472351990036
occupancy loss: 0.0

validation 

heatmap loss: 0.2799489329326325
count loss: 0.03568936330301566
occupancy loss: 0.0
training time: 4.0h 21m 34s

Epoch 36/75
----------

training 

heatmap loss: 0.634776809915305
count loss: 0.07781899194564529
occupancy loss: 0.0

validation 

heatmap loss: 0.28000863116699026
count loss: 0.036362929284634166
occupancy loss: 0.0
training time: 4.0h 29m 5s

Epoch 37/75
----------

training 

heatmap loss: 0.6272724303904499
count loss: 0.0720696101271423
occupancy loss: 0.0

validation 

heatmap loss: 0.27995948440344315
count loss: 0.042190957215091134
occupancy loss: 0.0
training time: 4.0h 36m 33s

Epoch 38/75
----------

training 

heatmap

heatmap loss: 0.6237247720532845
count loss: 0.04873261034836783
occupancy loss: 0.0

validation 

heatmap loss: 0.27961048945523365
count loss: 0.022835309796025886
occupancy loss: 0.0
training time: 8.0h 20m 33s

Epoch 68/75
----------

training 

heatmap loss: 0.64386030873318
count loss: 0.051449323147144385
occupancy loss: 0.0

validation 

heatmap loss: 0.2794380935654931
count loss: 0.024206644212897626
occupancy loss: 0.0
training time: 8.0h 28m 1s

Epoch 69/75
----------

training 

heatmap loss: 0.6382863281556812
count loss: 0.05024266281633755
occupancy loss: 0.0

validation 

heatmap loss: 0.2795118057046753
count loss: 0.02807446807453617
occupancy loss: 0.0
training time: 8.0h 35m 26s

Epoch 70/75
----------

training 

heatmap loss: 0.632755361239653
count loss: 0.04902306248320416
occupancy loss: 0.0

validation 

heatmap loss: 0.27953393289684747
count loss: 0.022247084599208034
occupancy loss: 0.0
training time: 8.0h 42m 53s

Epoch 71/75
----------

training 

heatma

### Validation<a name="2.1V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [59]:
# iterate over combinations
for row in combinations_21.iterrows():
    
   # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run training
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/Heatmap/UnetCntWRN_ts-vanilla/'

## Pipeline 2.2 - Heatmap + occupancy <a name="1.2"></a>
---

Heatmap + occupancy models, like pure Heatmap models, work by using semantic segmentation to generate a pixel-wise probability of a cell being a seal centroid. With a matrix of probabilities, we obtain a count by applying a sigmoid transform to it, thresholding and adding over all cells. After addining up cells, however, we use the output from a specialized occupancy branch -- and another threshold -- to decide if predicted counts will be set to zero (i.e. image is not occupied). Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image.

### Training<a name="2.2T"></a>

The following cell trains heatmap + occupancy based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [73]:
# switch pipeline
pipeline = 'Heatmap-Occ'

# generate model combinations
combinations_22 = {'model_architecture': ['UnetOccDense'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['D'] }       

# read as a DataFrame
combinations_22 = pd.DataFrame(combinations_22)
                    

# create folders for resulting files
for row in combinations_22.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]                  
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [74]:
# iterate over combinations
for row in combinations_22.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        #continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --output_name=$out  \
                             --models_folder=$dest_folder
   


training UnetOccDense_ts-vanilla

Epoch 1/75
----------

training 

heatmap loss: 2.4572517387924435
count loss: 16507.192337750817
occupancy loss: 0.7385156268601788

validation 

heatmap loss: 1.2295053821653026
count loss: 9790.048759090974
occupancy loss: 0.5099873172607899
training time: 0.0h 38m 4s

Epoch 2/75
----------

training 

heatmap loss: 0.981773634396474
count loss: 6166.263681978713
occupancy loss: 0.47164687838934954

validation 

heatmap loss: 0.5218042941549816
count loss: 3682.4144449888468
occupancy loss: 0.4790428731820187
training time: 1.0h 15m 20s

Epoch 3/75
----------

training 

heatmap loss: 0.570832517486243
count loss: 2454.899532418489
occupancy loss: 0.3629776326328483

validation 

heatmap loss: 0.30300493237073195
count loss: 1604.9769085666744
occupancy loss: 0.3409179259360116
training time: 1.0h 32m 58s

Epoch 4/75
----------

training 

heatmap loss: 0.42829849774640216
count loss: 1085.6956207543055
occupancy loss: 0.31272509321560815

validati

heatmap loss: 0.3180343539328354
count loss: 7.921611928256678
occupancy loss: 0.14507763758433964

validation 

heatmap loss: 0.140854105438162
count loss: 5.154737888631236
occupancy loss: 0.3818798451752668
training time: 6.0h 49m 39s

Epoch 32/75
----------

training 

heatmap loss: 0.31837015683542613
count loss: 7.714475379552753
occupancy loss: 0.13900540896259653

validation 

heatmap loss: 0.1404479663607566
count loss: 7.094378158809232
occupancy loss: 0.3959242865891958
training time: 7.0h 0m 50s

Epoch 33/75
----------

training 

heatmap loss: 0.31209157793045217
count loss: 6.7884156002840506
occupancy loss: 0.13677654177182189

validation 

heatmap loss: 0.14027274040010076
count loss: 4.679278138815495
occupancy loss: 0.3190218966831063
training time: 7.0h 11m 58s

Epoch 34/75
----------

training 

heatmap loss: 0.324382920483999
count loss: 7.388021512888732
occupancy loss: 0.1432709553714689

validation 

heatmap loss: 0.14047356527533283
count loss: 4.43709694304979

heatmap loss: 0.3135706369405715
count loss: 4.66503304701019
occupancy loss: 0.10923125301315512

validation 

heatmap loss: 0.14019907015227875
count loss: 2.898622340091139
occupancy loss: 0.8210207112912916
training time: 12.0h 23m 18s

Epoch 62/75
----------

training 

heatmap loss: 0.3150334488465869
count loss: 4.998856203635597
occupancy loss: 0.11360226072190006

validation 

heatmap loss: 0.14038689209708108
count loss: 6.542809699909769
occupancy loss: 0.45676182976397217
training time: 12.0h 34m 25s

Epoch 63/75
----------

training 

heatmap loss: 0.3155953310001362
count loss: 4.824436133509877
occupancy loss: 0.11527228902711988

validation 

heatmap loss: 0.13993830141615246
count loss: 2.432447733834218
occupancy loss: 0.48488403498061794
training time: 12.0h 45m 32s

Epoch 64/75
----------

training 

heatmap loss: 0.31302301904286783
count loss: 4.693562596413164
occupancy loss: 0.10585623757463714

validation 

heatmap loss: 0.1400026468651105
count loss: 1.9362737

### Validation<a name="2.2V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [75]:
# iterate over combinations
for row in combinations_22.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run validation
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder


validating UnetOccDense_ts-vanilla

Traceback (most recent call last):
  File "validate_sealnet.py", line 295, in <module>
    main()
  File "validate_sealnet.py", line 289, in main
    val_stats = validate_model(model, criterion, criterion2, criterion3, bce_weight=bce_weights)
  File "validate_sealnet.py", line 157, in validate_model
    out_dict = model(inputs)
  File "/home/bento/anaconda3/envs/geospatial/lib/python3.7/site-packages/torch/nn/modules/module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/bento/anaconda3/envs/geospatial/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py", line 152, in forward
    outputs = self.parallel_apply(replicas, inputs, kwargs)
  File "/home/bento/anaconda3/envs/geospatial/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py", line 162, in parallel_apply
    return parallel_apply(replicas, inputs, kwargs, self.device_ids[:len(replicas)])
  File "/home/bento/anaconda3/envs/geospatial/li

# Pipeline 3 - Heatmap + Count + Occupancy<a name="1.3T"></a>

Like all previous ones, this pipeline will also generate a seal intensity heatmap. Similar to Heatmap + count models, models on this pipeline have a specialized branch to get a count by regression. After counting, however, we use a threshold and the output from a specialized occupancy branch to decide if we will set the count to zero (i.e. image is not occupied). Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image


### Training<a name="3T"></a>

The following cell trains heatmap + count + occupancy based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [19]:
dest_folder = 'saved_models_all_train'
# switch pipeline
pipeline = 'Heatmap-Cnt-Occ'

# generate model combinations
combinations_3 = {'model_architecture': ['UnetCntWRNOccDense'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['F'] }       

# read as a DataFrame
combinations_3 = pd.DataFrame(combinations_3)
                    

# create folders for resulting files
for row in combinations_3.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]                  
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [ ]:
# iterate over combinations
for row in combinations_3.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        #continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --output_name=$out  \
                             --models_folder=$dest_folder
   


training UnetCntWRNOccDense_ts-vanilla

Epoch 1/75
----------

training 

heatmap loss: 1.1644826078369537
count loss: 0.44770444824162137
occupancy loss: 0.612192434933626

validation 

heatmap loss: 0.31206839670685144
count loss: 0.17629554726154345
occupancy loss: 0.3594160751654025
training time: 0.0h 12m 23s

Epoch 2/75
----------

training 

heatmap loss: 0.6579567318195184
count loss: 0.3318426635350804
occupancy loss: 0.3827065152363257

validation 

heatmap loss: 0.29042118666695704
count loss: 0.09255690026206655
occupancy loss: 0.1516428045981489
training time: 0.0h 24m 42s

Epoch 3/75
----------

training 

heatmap loss: 0.6590061643360395
count loss: 0.24069830167001574
occupancy loss: 0.289599810652706

validation 

heatmap loss: 0.2852072783852488
count loss: 0.1652639008817362
occupancy loss: 0.622003287344569
training time: 0.0h 39m 55s

Epoch 4/75
----------

training 

heatmap loss: 0.6422445176974095
count loss: 0.19618477468496243
occupancy loss: 0.24505671833415

### Validation<a name="3V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [44]:
# iterate over combinations
for row in combinations_3.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run validation
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder


validating UnetCntWRNOccDense_ts-vanilla

heatmap loss: 0.10520597549482344
count loss: 0.02247610786535993
occupancy loss: 0.06235330289481412
Validation complete in 0.0h 1m 36s
